# Youtube Comments Advanced Analysis 🎥

🎯 With this project, I wanted to use NLP techniques to analyze large amounts of social media comments from Youtube. My theory is that tasks like sentiment analysis and toxicity detection provide a high level view and an empirical representation of textual engagement.

Large YouTube channels receive such a high number of comments that it becomes infeasible to read them all especially as new comments are posted. Such a reading and assessment would additionally be qualitative in nature. It also seems that creators engage with the comment section soon after a video is published, but neglect the wealth of information found there as time passes. Creators therefore have a tendency to view and respond to early feedback while not considering recent comments.

This type of project takes steps to provide insights into the comment section to potentially enable creator decision making.

**Outline**
 1. Read Data via Pandas
 2. Preprocessing
 3. Add Sentiment Analysis
 4. Add Feedback comments
 5. Add toxic comment identifier
 6. Add Streamlit dashboard


## Libraries 📚

In [ ]:
from IPython.display import clear_output

In [ ]:
%pip install transformers streamlit emoji -q

In [ ]:
!pip install wordcloud localtunnel -q

In [ ]:
# Import necessary libraries
import pandas as pd
from transformers import pipeline
import streamlit as st
import emoji
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os
import numpy as np
import torch

# Make device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

## Read Data 📖

The script mounts to Google Drive. Using Pandas, it reads in the two csv files handling errors and removes duplicates.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

dir = '/content/drive/MyDrive/Python Scripts/YT Comments Analytics'

Mounted at /content/drive


In [ ]:
# 1. Read Data via Pandas
# Read the first comments and video data
US_comments = pd.read_csv(os.path.join(dir, "UScomments.csv"), error_bad_lines=False)
US_videos = pd.read_csv(os.path.join(dir, "USvideos.csv"), error_bad_lines=False)
# skipping bad lines because the data was formated in such a way that I couldn't identify how to reintegrate it.
# Also, the loss of 17 rows of data shouldn't be detremental to the overall system performance.

# Read the second comments and video data
GB_comments = pd.read_csv(os.path.join(dir, "GBcomments.csv"), error_bad_lines=False)
GB_videos = pd.read_csv(os.path.join(dir, "GBvideos.csv"), error_bad_lines=False)

# Concatenate (union) the comments dataframes
comment_data = pd.concat([US_comments, GB_comments], ignore_index=True)

# Concatenate (union) the videos dataframes
video_data = pd.concat([US_videos, GB_videos], ignore_index=True)

# Save the concatenated dataframes to new CSV files
comment_data.to_csv("all_comments.csv", index=False)
video_data.to_csv("all_videos.csv", index=False)

# view comment duplicates
# duplicates_mask = comment_data.duplicated(subset=['video_id', 'comment_text'], keep=False)
# duplicates = comment_data[duplicates_mask]
# print(duplicates.sort_values(['video_id', 'comment_text']))

comment_data = comment_data.drop_duplicates(subset=['video_id', 'comment_text'], keep='first')

# view video duplicates
# duplicates_mask = video_data.duplicated(subset=['video_id'], keep=False)
# duplicates = video_data[duplicates_mask]
# print(duplicates.sort_values(['video_id']))

video_data = video_data.sort_values(by='views', ascending=False).drop_duplicates(subset=['video_id'], keep='first')


<ipython-input-20-e5abee96edc9>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  US_comments = pd.read_csv(os.path.join(dir, "UScomments.csv"), error_bad_lines=False)
Skipping line 41589: expected 4 fields, saw 11
Skipping line 51628: expected 4 fields, saw 7
Skipping line 114465: expected 4 fields, saw 5

Skipping line 142496: expected 4 fields, saw 8
Skipping line 189732: expected 4 fields, saw 6
Skipping line 245218: expected 4 fields, saw 7

Skipping line 388430: expected 4 fields, saw 5

<ipython-input-20-e5abee96edc9>:3: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  US_comments = pd.read_csv(os.path.join(dir, "UScomments.csv"), error_bad_lines=False)
<ipython-input-20-e5abee96edc9>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  US_vid

In [ ]:
# Preview the data to verify there won't be issues later
comment_data.head()
comment_data.tail()
video_data.head()
video_data.tail()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
15980,T2RUYYs8Hxc,DJ Cummerbund - Hey Trump!,DJ Cummerbund,10,eminem|freestyle|bet|rage against the machine|...,118453,1846,92,240,https://i.ytimg.com/vi/T2RUYYs8Hxc/default.jpg,22.1
15981,KkQdnJK1iF8,Emotional Groom Start Crying When He Sees Brid...,Today`s Video,22,Today's Video|Today Video|Latest News|Last|New...,76096,194,19,31,https://i.ytimg.com/vi/KkQdnJK1iF8/default.jpg,22.1
15982,1zOPtQNChZM,FALLING IN LOVE WITH SANTORINI,Estée Lalonde,22,essiebutton|Estée Lalonde|Estee Lalonde|Essie ...,170003,6592,127,454,https://i.ytimg.com/vi/1zOPtQNChZM/default.jpg,22.1
15983,Z7R8XRKqHAI,Hey Ya! - Walk off the Earth (Outkast Cover),Walk off the Earth,10,hey ya cover|amazing cover|Shape of You Walk o...,893125,44811,652,2056,https://i.ytimg.com/vi/Z7R8XRKqHAI/default.jpg,22.1
15984,lLN1FwiqGwc,It (2017) - Nostalgia Critic,Channel Awesome,24,channel awesome|nostalgia critic|doug walker|m...,1474925,58532,3223,14396,https://i.ytimg.com/vi/lLN1FwiqGwc/default.jpg,22.1


## Preprocessing 🔧

Here I filter the data by comparing it to my personal list of subscriptions. To include additional channels, I create a popularity index which I use to select additional top channels. By filtering the data, processing speeds increase and it focuses the analysis somewhat.

I also replace the return character with a space, lower the case of the comment text, and replace emojis with text descriptions.

In [ ]:
# 2. Preprocessing
# Filter the data to subset

# Include any channels from my personal list of subscriptions also in the data
subs = pd.read_csv(os.path.join(dir,'subscriptions.csv'))
sub_channels = subs['Channel Title'].unique().tolist()

# dataset channels
ds_channels = video_data.channel_title.unique().tolist()
common = list(set(ds_channels) & set(sub_channels))
common

['Star Wars',
 'Foil Arms and Hog',
 'Mark Rober',
 'Vox',
 'Kurzgesagt – In a Nutshell',
 '3Blue1Brown',
 'CrashCourse']

In [ ]:
# Take the 10 most popular channels (not already included) based on a popularity index
# define popularity index based on: 'views', 'likes, 'comment_total', & -'dislikes'
video_data['popularity_index'] = video_data['views'] + video_data['likes'] + video_data['comment_total'] - video_data['dislikes']

# Group by the 'Group' column and calculate the sum within each group
grouped_vd = video_data.groupby('channel_title')['popularity_index'].sum().reset_index()

# Rank the groups based on the sum in descending order
grouped_vd['Rank'] = grouped_vd['popularity_index'].rank(ascending=False, method='min')

top_channels = grouped_vd.sort_values(by='Rank').head(3).channel_title.tolist() # there are overlapping videos between my subs and popular channels
top_channels

['ibighit', 'SamSmithWorldVEVO', 'BuzzFeedVideo']

In [ ]:
# combine the lists and filter the dataset
channels = list(set(common + top_channels))
assert len(channels) == 10

# filter the video data
filtered_vd = video_data[video_data.channel_title.isin(channels)]
filtered_vd.to_csv(os.path.join(dir,'video_processed_checkpoint.csv'), index=False)
# filter the comment data
video_ids = filtered_vd.video_id.unique().tolist()
comment_data = comment_data[comment_data.video_id.isin(video_ids)]
comment_data.shape

(17810, 4)

In [ ]:
# Change emoji's into text description within paranthesis
def replace_emoji(text: str):
    return emoji.demojize(str(text))
    # Note: this leaves colons at the beginning and en dof the description text
    # Note2: multiword descriptions are joined by an underscore

comment_data['comment_text'] = comment_data['comment_text'].apply(replace_emoji)
video_data['title'] = video_data['title'].apply(replace_emoji)

# Normalize by lowering the case and replacing \n with a space
comment_data['comment_text'] = comment_data['comment_text'].str.lower()
comment_data['comment_text'] = comment_data['comment_text'].str.replace('\n', ' ')
video_data['title'] = video_data['title'].str.replace('\n', ' ')

## Sentiment Analysis 🔼🔽🤷

Here, I used the `lxyuan/distilbert-base-multilingual-cased-sentiments-student` model to output the sentiment of a comment (Positive, Negative, or Neutral).

In [ ]:
# 3. Add Sentiment Analysis via HuggingFace model pipeline
sentiment_pipeline = pipeline("sentiment-analysis",
                              "lxyuan/distilbert-base-multilingual-cased-sentiments-student",
                              device=device)

def get_sentiment(text):
    try:
      result = sentiment_pipeline(text)
      return result[0]['label']
    except:
      print(text)
      print(len(text))
      return np.nan


comment_data['sentiment'] = comment_data['comment_text'].apply(get_sentiment)
# TODO explore getting specific emotions instead of just pos, neg, & neu

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (675 > 512). Running this sequence through the model will result in indexing errors


:purple_heart::purple_heart::purple_heart::purple_heart::face_blowing_a_kiss::face_blowing_a_kiss::face_blowing_a_kiss::face_blowing_a_kiss::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::face_blowing_a_kiss::face_blowing_a_kiss::face_blowing_a_kiss::purple_heart::purple_heart::purple_heart::purple_heart::purple_heart::purple_heart::face_blowing_a_kiss::face_blowing_a_kiss::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::sad_but_relieved_face::sad_but_relieved_face::sad_but_relieved_face::frowning_face_with_open_mouth::frowning_face_with_open_mouth::frowning_face_with_open_mouth::loudly_crying_face::loudly_crying_face::kiss_mark::kiss_mark::kiss_mark::blue_heart::blue_heart::blue_heart::blue_heart::blue_heart::blue_heart::blue_heart::red_heart::red_heart::red_heart::red_heart::red_heart::red_heart::purple_heart::purple_heart::purple_heart::yel

In [ ]:
# check point
comment_data.to_csv(os.path.join(dir,"processed_comments_sentiment_checkpoint.csv"), index=False)

In [ ]:
# load check point
# comment_data = pd.read_csv(os.path.join(dir,"processed_comments_sentiment_checkpoint.csv"))

<ipython-input-18-d657d739f31d>:2: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  comment_data = pd.read_csv(os.path.join(dir,"processed_comments_sentiment_checkpoint.csv"))


## Identify Suggestions 🗣📢

I used ChatGPT to generate a list of phrases used to indicate suggestions (i.e. "You should", "I recommend", etc.). Then taking a heuristic approach, I check if those phrases are in the comment and mark it as a suggestion.

In [ ]:
# 4. Add comments with suggestions
# Establish heurstic baseline
def txt_to_list(file_path):
    """
    Reads a text file and extracts content between quotation marks to form a Python list of strings.

    Parameters:
    - file_path (str): The path to the text file.

    Returns:
    - list: A Python list of strings.
    """
    try:
        with open(file_path, 'r') as file:
            content = file.readlines()
            # Extract content between quotation marks, excluding the last character
            string_list = [line.strip()[1:-2].lower() for line in content]
        return string_list
    except FileNotFoundError:
        print(f"Error: File not found at path '{file_path}'")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [ ]:
# read and normalize the text file of suggestion phrases
phrases = os.path.join(dir,"Suggestion_Phrases.txt")
phr_list = txt_to_list(phrases)
phr_list[:5]

['you should',
 'i recommend',
 'have you thought to try',
 'consider trying',
 'i suggest']

In [ ]:
def check_contains_suggestion(comment_text, suggestion_phrases: list):
    """
    Check if any of the specified phrases are present in the comment_text.

    Parameters:
    - comment_text (str): String containing comment text.
    - suggestion_phrases (list): List of phrases to check for in the comments.

    Returns:
    - bool: True if any suggestion phrase is present, False otherwise.
    """
    # Check if any suggestion phrase is present in the comment_text
    return any(phrase.lower() in comment_text.lower() for phrase in suggestion_phrases)

In [ ]:
comment_data['contains_suggestion'] = comment_data['comment_text'].apply(lambda x: check_contains_suggestion(x, phr_list))
comment_data.head()

,video_id,comment_text,likes,replies,sentiment,contains_suggestion
3198,GGm0FQ6i74U,this is a great interview for people to watch ...,4,0,positive,False
3199,GGm0FQ6i74U,what i think: *this video is too damn loo0o0oo...,0,0,negative,False
3200,GGm0FQ6i74U,"who cares what she thinks?\nbesides, we alread...",0,0,negative,False
3201,GGm0FQ6i74U,what she's thinking is doesn't matter..what's ...,0,0,neutral,False
3202,GGm0FQ6i74U,it should say what a narcissist really thinks,0,0,negative,False


In [ ]:
comment_data[comment_data['contains_suggestion'] == True]

,video_id,comment_text,likes,replies,sentiment,contains_suggestion
3206,GGm0FQ6i74U,why didn't you ask her about all the crooked ...,0,0,negative,True
18347,YSVlSmZWzm0,maybe you shouldn't have moved so many times.,0,0,neutral,True
18410,YSVlSmZWzm0,if your looking for a interesting and (i guess...,0,0,negative,True
38113,YSVlSmZWzm0,how about the illiad by homer?,0,0,negative,True
46024,YSVlSmZWzm0,how about the illiad by homer?,0,0,negative,True
...,...,...,...,...,...,...
1374071,GoJsr4IwCm4,watch part 2 by cgpgrey: https://www.youtube.c...,2392,200,negative,True
1392327,GoJsr4IwCm4,watch part 2 by cgpgrey: https://www.youtube.c...,2980,239,negative,True
1392335,GoJsr4IwCm4,how about just live for jesus and actually hav...,0,0,negative,True
1392352,GoJsr4IwCm4,"hey, i loved the video, but you shouldn't sell...",0,0,negative,True


In [ ]:
# check point
comment_data.to_csv(os.path.join(dir,"processed_comments_suggestion_checkpoint.csv"), index=False)

## Identify Toxic Comments ☢💀☢

Social media is plagued by toxicity. I use the `Xuhui/ToxDect-roberta-large` model to identify a comment as toxic or non-toxic.

In [ ]:
# 5. Add toxic comment identifier via HuggingFace model pipeline
# toxicity_pipeline = pipeline("text-classification", model="longluu/distilbert-toxic-comment-classifier")
toxicity_pipeline = pipeline("text-classification",
                             model="Xuhui/ToxDect-roberta-large",
                             device=device)
# toxicity_pipeline = pipeline("text-classification",
#                              model="martin-ha/toxic-comment-model",
#                              device=device)

# todo find a smaller model

def is_toxic(text):
  try:
    result = toxicity_pipeline(text)
    return result[0]['label'] == 'LABEL_1'
  except:
    # Split the input text into sections
    section_length = 500
    sections = [text[i:i+section_length] for i in range(0, len(text), section_length)]

    print(text)
    print(len(text))

    # Loop through each section and apply sentiment pipeline
    for section in sections:
        result = sentiment_pipeline(section)
        if result[0]['label'] == 'LABEL_1':
            print('True')
            return True
    print('False')
    return False

comment_data['is_toxic'] = comment_data['comment_text'].apply(is_toxic)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (573 > 512). Running this sequence through the model will result in indexing errors


:purple_heart::purple_heart::purple_heart::purple_heart::face_blowing_a_kiss::face_blowing_a_kiss::face_blowing_a_kiss::face_blowing_a_kiss::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::face_blowing_a_kiss::face_blowing_a_kiss::face_blowing_a_kiss::purple_heart::purple_heart::purple_heart::purple_heart::purple_heart::purple_heart::face_blowing_a_kiss::face_blowing_a_kiss::smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::sad_but_relieved_face::sad_but_relieved_face::sad_but_relieved_face::frowning_face_with_open_mouth::frowning_face_with_open_mouth::frowning_face_with_open_mouth::loudly_crying_face::loudly_crying_face::kiss_mark::kiss_mark::kiss_mark::blue_heart::blue_heart::blue_heart::blue_heart::blue_heart::blue_heart::blue_heart::red_heart::red_heart::red_heart::red_heart::red_heart::red_heart::purple_heart::purple_heart::purple_heart::yel

In [ ]:
comment_data[comment_data['is_toxic'] == True]

,video_id,comment_text,likes,replies,sentiment,contains_suggestion,is_toxic
3199,GGm0FQ6i74U,what i think: *this video is too damn loo0o0oo...,0,0,negative,False,True
3205,GGm0FQ6i74U,wow \nwhat a liar!!!!! \nhow anyone can believ...,0,0,negative,False,True
3206,GGm0FQ6i74U,why didn't you ask her about all the crooked ...,0,0,negative,True,True
3210,GGm0FQ6i74U,you had no message lady not to mention you are...,0,0,negative,False,True
3214,GGm0FQ6i74U,people forget one little major thing. the demo...,0,0,negative,False,True
...,...,...,...,...,...,...,...
1409092,LunHybOKIjU,you need white friends bro,0,0,positive,False,True
1409093,LunHybOKIjU,what a talentless knob end,0,0,positive,False,True
1409094,LunHybOKIjU,faginem,0,0,negative,False,True
1409095,LunHybOKIjU,garbage! #maga2020:united_states::red_heart:,0,0,positive,False,True


In [ ]:
# checkpoint
comment_data.to_csv(os.path.join(dir,"processed_comments_toxicity_checkpoint.csv"))

In [ ]:
# load check point
comment_data = pd.read_csv(os.path.join(dir,"processed_comments_toxicity_checkpoint.csv"))

## Streamlit Dashboard 📊

This section creates a Streamlit dashboard that loads the processed YouTube comment and video data, reintroduces emojis, merges the datasets, and provides interactive filters for channel and video title selection. The dashboard includes visualizations such as sentiment distribution and toxic comments percentage, along with total engagement and toxic comments counts. Additionally, it showcases suggestion comments, lists randomly sampled comments, displays a word cloud of commonly used words. The script creates an informative interface for exploring YouTube comments.

In [ ]:
# 6. Add Streamlit dashboard
%%writefile streamlit.py
import pandas as pd
import streamlit as st
import os
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import emoji

dir = '/content/drive/MyDrive/Python Scripts/YT Comments Analytics'

# Load checkpoint
comment_data = pd.read_csv(os.path.join(dir, "processed_comments_toxicity_checkpoint.csv"))
video_data = pd.read_csv(os.path.join(dir, "video_processed_checkpoint.csv"))

# Add emoji back in
def add_emojis(text):
    return emoji.emojize(text)

comment_data['comment_text'] = comment_data['comment_text'].apply(add_emojis)
video_data['title'] = video_data['title'].apply(add_emojis)

# Add Channel Title to the comment_data
comment_data = comment_data.merge(video_data, how='left', on='video_id')

# 1. Add a filter for video and channel title
selected_channel = st.sidebar.selectbox("Select Channel", ['All'] + list(comment_data['channel_title'].unique()))

# Filter video titles based on the selected channel
available_video_titles = ['All'] + list(comment_data[comment_data['channel_title'] == selected_channel]['title'].unique())
selected_video = st.sidebar.selectbox("Select Video", available_video_titles)

# Apply filters
filtered_comments = comment_data.copy()
if selected_channel != 'All':
    filtered_comments = filtered_comments[filtered_comments['channel_title'] == selected_channel]
if selected_video != 'All':
    filtered_comments = filtered_comments[filtered_comments['title'] == selected_video]


st.title("YouTube Comments Analysis")

# 2. Sort the bar chart for Positive, Negative, and Neutral
#    Each category should have a color, and display percentages
sentiment_chart = filtered_comments['sentiment'].value_counts(normalize=True).sort_index()
sentiment_chart *= 100  # Convert to percentage
sentiment_chart = sentiment_chart.round(2)
# colors = {'positive': 'green', 'negative': 'red', 'neutral': 'blue'}
colors = '#99C3EC' # ['97E3C2','F69C9E','99C3EC']
# st.bar_chart(sentiment_chart, color=sentiment_chart.index.map(colors))
st.bar_chart(sentiment_chart, color=colors)
st.text("Sentiment Distribution")

# 3. Change the format of Total Engagement & Total Toxic Comments
total_engagement = video_data[['likes', 'dislikes']].sum().sum()
total_toxic_comments = filtered_comments['is_toxic'].sum()
st.write(f"**Total Engagement:** {total_engagement:,}")
st.write(f"**Total Toxic Comments:** {total_toxic_comments:,}")

# 4. The chart for what percentage of comments were Toxic, Not Toxic, and NaN
toxic_percentage = filtered_comments['is_toxic'].value_counts(normalize=True) * 100
toxic_percentage = toxic_percentage.round(2)
toxic_colors = {'True': 'red', 'False': 'green'}
toxic_color = '#F69C9E'
# st.bar_chart(toxic_percentage, color=toxic_percentage.index.map(toxic_colors))
st.bar_chart(toxic_percentage, color=toxic_color)
st.text("Toxic Comments Distribution")

# 5. Add section to display 5 random comments where contains_suggestion is True
suggested_comments = filtered_comments[(filtered_comments['contains_suggestion'] == True) & (filtered_comments['sentiment'].isin(['positive','neutral']))]
try:
  suggested_comments = suggested_comments.sample(5)
except:
  suggested_comments = suggested_comments
st.subheader("Sample Comments with Suggestions:")
st.table(suggested_comments[['comment_text', 'sentiment', 'is_toxic']])

# 6. Add the contains_suggestion column to the Sample Comments
st.subheader("Sample Comments:")
st.table(filtered_comments[['comment_text', 'sentiment', 'is_toxic', 'contains_suggestion']].head(10))

# 7. Display Word Cloud of comments
# wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(filtered_comments['comment_text']))
# plt.figure(figsize=(10, 5))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis('off')
# fig, ax = plt.subplots()
# st.pyplot(fig)

# 7. Display Word Cloud of comments
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(filtered_comments['comment_text']))

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 5))

# Display the word cloud on the axis
ax.imshow(wordcloud, interpolation='bilinear')
ax.axis('off')

# Show the figure using st.pyplot()
st.pyplot(fig)

Overwriting streamlit.py


In [ ]:
!streamlit run streamlit.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule openurl@1.1.1 c


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.224.102.71:8501

npx: installed 22 in 2.927s
your url is: https://pink-ducks-grab.loca.lt
  Stopping...
^C


## Potential Additional Features 🌆

I would love to pour my heart and soul into perfecting this project. Maybe one day I'll have the time. Here are some additional features that could be added.

- Sentiment analysis could include the specific emotion expressed to understand the overall emotion of the section
- We could include a feature for bot detection, as well.
- There is more work to do on the Streamlit UI.
- Room for more analytics (What video generated the most toxic comments?, etc.)
- Breakout types of engagement into a chart
- Allow for comparison between videos
- Add language detection as a feature
- Add machine translation (MT) functionality
- Refine selected models
  - Sentiment leans negative and is by no means perfect.
  - Toxicity is also not perfect, but catches important issues.


## Thoughts 💭
- Just from a cursory glance of the data, technical videos generate a more positive response while opinion/explainer type video generate a lot more negativity.
- There is still much more analysis to perform.

This was a fun way to shed some light on what's happening in the comment section using some advanced NLP analysis.

Thanks for coming to my TED Talk!

## Citations

Dataset: https://www.kaggle.com/datasets/datasnaek/youtube?resource=download

Sentiment Analysis Model: https://huggingface.co/lxyuan/distilbert-base-multilingual-cased-sentiments-student

Toxicity Detection Model: https://huggingface.co/Xuhui/ToxDect-roberta-large

Note: GPT-3.5-turbo was used for refactoring & function generation throughout this project while the structure and ideas are entirely mine.